<a href="https://colab.research.google.com/github/saifulrijal-ds/image-classification-with-DL/blob/main/Keras_CIFAR10_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pada latihan ini kita akan melakukan klasifikasi multi-kelas yaitu mengklasifikasikan gambar ke dalam 10 kelas dengan menggunakan dataset CIFAR10

In [3]:
# set random seed
from tensorflow.random import set_seed
set_seed(404)

In [4]:
# import package
import numpy as np
import tensorflow as tf
from tensorflow import keras